Для слова `Острый` я выбрала значения 1 и 6 из [МАС](http://feb-web.ru/feb/mas/mas-abc/default.asp) и скачала из НКРЯ два датасета соответственно:

1. Пища и напитки (семантические признаки)
2. инструменты и оружие

Обработаем датасеты.

In [21]:
import adagram
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from tqdm.notebook import tqdm
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

In [22]:
import pandas as pd

sharp_dataset = pd.read_csv("sharp.csv", sep="\t", error_bad_lines=False)
spicy_dataset = pd.read_csv("spicy.csv", sep="\t", error_bad_lines=False)

b'Skipping line 47: expected 19 fields, saw 28\nSkipping line 216: expected 19 fields, saw 22\nSkipping line 217: expected 19 fields, saw 27\nSkipping line 262: expected 19 fields, saw 21\nSkipping line 285: expected 19 fields, saw 22\nSkipping line 317: expected 19 fields, saw 23\nSkipping line 343: expected 19 fields, saw 27\nSkipping line 399: expected 19 fields, saw 22\nSkipping line 410: expected 19 fields, saw 25\nSkipping line 425: expected 19 fields, saw 21\nSkipping line 439: expected 19 fields, saw 21\n'


In [23]:
sharp_dataset.head(5)

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,"етинкторп ,вотог ьсуг отч ,ясьтидебу",мыртсо,"убедиться, что гусь готов, проткните",острым,,ножом бедро.,Рецепты национальных кухонь: Скандинавская кух...,NaN,NaN,Рецепты национальных кухонь: Скандинавская кухня,2000-2005,"бытовая, нехудожественная, производственно-тех...",рецепт,дом и домашнее хозяйство,NaN,NaN,электронный текст,омонимия снята,"Для того, чтобы убедиться, что гусь готов, про..."
1,йошьлоб и ― иворк в,йыртсо,в крови ― и большой,острый,,нож.,Запись LiveJournal (2004),NaN,NaN,Запись LiveJournal,2004,"нехудожественная, электронная коммуникация",блог,частная жизнь,NaN,NaN,электронный текст,омонимия снята,"В руках у безумца ― портфель, в котором лежат ..."
2,тюазерорп и укйенил тюавыдалкирп метаЗ,мыртсо,Затем прикладывают линейку и прорезают,острым,,ножом уложенные внахлест оба полотнища.,Елена Волкова. Тот самый линолеум // «Биржа пл...,Елена Волкова,NaN,Тот самый линолеум,2002,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Биржа плюс свой дом» (Н. Новгород),2002.08.12,газета,омонимия снята,Затем прикладывают линейку и прорезают острым ...
3,"хынназероп ,хывенчирок хитэ ан лавымердоп",имыртсо,"подремывал на этих коричневых, порезанных",острыми,,"ножиками сиденьях, в головах тоже",Андрей Волос. Недвижимость (2000) // «Новый Ми...,Андрей Волос,1955,Недвижимость,2000,художественная,роман,_,«Новый Мир»,2001,журнал,омонимия снята,"Электричка покачивалась, и, наверное, у всех, ..."
4,ёе илырку ино адог авд,еыртсо,два года они укрыли её,острые,,лопатки.,Людмила Улицкая. Казус Кукоцкого [Путешествие ...,Людмила Улицкая,1943,Казус Кукоцкого [Путешествие в седьмую сторону...,2000,художественная,роман,_,«Новый Мир»,2000,журнал,омонимия снята,Теперь Таня носила короткие чёрные платья и дл...


In [61]:
from pymystem3 import Mystem
mystem = Mystem()

class WSD:
    def __init__(self, text, meaning):
        self.text = text
        self.word = "острый"
        self.meaning = meaning
        self.neighbours = None
        self.disambiguated_meaning = None
        
    def __repr__(self):
        return f"WSD(text={self.text}, meaning={self.meaning})"
    
    @staticmethod
    def lemmatized_context(s):
        return [w.lower() for w in mystem.lemmatize(" ".join(token.tokenize(s)))]
    
    def disambiguate(self, model):
        probs = model.disambiguate(self.word, self.lemmatized_context(self.text))
        self.disambiguated_meaning = probs.argmax()
        self.neighbours = model.sense_neighbors(self.word, np.argmax(probs))
        return self

В Адаграме значение "острая пища" имеет индекс 0, а "острый нож" - 3.

In [62]:
full_data = []

for i in range(len(sharp_dataset['Center'])):
    full_data.append(WSD(" ".join([sharp_dataset["Left context"][i].strip(), 
                     sharp_dataset["Center"][i].strip(), 
                     sharp_dataset["Right context"][i].strip()]),
                     3))
    
for i in range(len(spicy_dataset['Center'])):
    full_data.append(WSD(" ".join([spicy_dataset["Left context"][i].strip(), 
                     spicy_dataset["Center"][i].strip(), 
                     spicy_dataset["Right context"][i].strip()]),
                     0))

In [63]:
len(full_data)

610

In [64]:
for i in range(10):
    print(full_data[i])

WSD(text=убедиться, что гусь готов, проткните острым ножом бедро., meaning=3)
WSD(text=в крови ― и большой острый нож., meaning=3)
WSD(text=Затем прикладывают линейку и прорезают острым ножом уложенные внахлест оба полотнища., meaning=3)
WSD(text=подремывал на этих коричневых, порезанных острыми ножиками сиденьях, в головах тоже, meaning=3)
WSD(text=два года они укрыли её острые лопатки., meaning=3)
WSD(text=Высоченная, с острыми пиками поверху железная огорожа была, meaning=3)
WSD(text=человека с безумными глазами и острым ножом в руках., meaning=3)
WSD(text=как бы металлический браслет с острыми крючьями внутри., meaning=3)
WSD(text=Далее: готовы ли клюквенный сок, острый нож, штопор для выковыривания глаз, шило, meaning=3)
WSD(text=поздно), подвесил на крюк, достал острый нож, зажмурился и вонзил нож, meaning=3)


In [8]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [65]:
accuracy_heap = []

for context in tqdm(full_data):
    context.disambiguate(vm)
    accuracy_heap.append(context.disambiguated_meaning == context.meaning)
    
print("Accuracy of Adagram:", sum(accuracy_heap) / len(accuracy_heap))

/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
/usr/local/lib/python3.6/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms



Accuracy of Adagram: 0.7049180327868853


В принципе, вышло неплохо. Учитывая скидку на то, что тестовый корпус не на 100% чистый (иногда там есть левые контексты), даже совсем хорошо. Посмотрим, что будет со словарными значениями из MAC:

In [47]:
spicy_wsd = WSD("С большим количеством соли, пряностей, специй (о пище). Острые блюда. Острый сыр. || Едкий, пряный (о приправах). Острый перец. Острая горчица.", 0)
sharp_wsd = WSD("Имеющий хорошо колющий конец или хорошо режущий край; противоп. тупой. Острая игла. Острая бритва. Острый нож. □ — Острые колючки рвали мою одежду. Лермонтов, Бэла. Они навязали их [крючки] на поводки из проволоки или струны, которых щуки не могли перекусить, несмотря на свои острые зубы. С. Аксаков, Детские годы Багрова-внука. Я растирал себе в кровь руки и колени о песок, об острые камни, о колючие сучки, но ничего не замечал. М. Пришвин, За волшебным колобком.", 3)

spicy_wsd.disambiguate(vm)
print("If guessed the proper meaning of 'spicy':", spicy_wsd.disambiguated_meaning == spicy_wsd.meaning)
sharp_wsd.disambiguate(vm)
print("If guessed the proper meaning of 'sharp':", sharp_wsd.disambiguated_meaning == sharp_wsd.meaning)

If guessed the proper meaning of 'spicy': True
If guessed the proper meaning of 'sharp': True


Даже если убрать из толкований примеры, все равно норм.

In [48]:
spicy_wsd_2 = WSD("С большим количеством соли, пряностей, специй (о пище)", 0)
sharp_wsd_2 = WSD("Имеющий хорошо колющий конец или хорошо режущий край; противоп. тупой.", 3)

spicy_wsd_2.disambiguate(vm)
print("If guessed the proper meaning of 'spicy':", spicy_wsd_2.disambiguated_meaning == spicy_wsd_2.meaning)
sharp_wsd_2.disambiguate(vm)
print("If guessed the proper meaning of 'sharp':", sharp_wsd_2.disambiguated_meaning == sharp_wsd_2.meaning)

If guessed the proper meaning of 'spicy': True
If guessed the proper meaning of 'sharp': True


Далее я хочу использовать информацию, полученную из sense_neighbours, чтобы соотнести толкования со значениями из адаграма. Я возьму список слов, гипонимов и гиперонимов для каждого синсета и буду искать пересечения с "соседями" из адаграма. Там, где пересечение будет наибольшим, там толкования + значения совпадают.

In [92]:
!wget https://github.com/sjut/HSE-Compling/raw/master/seminars/data/relations_with_concepts.csv

--2019-12-07 22:30:36--  https://github.com/sjut/HSE-Compling/raw/master/seminars/data/relations_with_concepts.csv
Распознаётся github.com (github.com)… 140.82.118.3
Подключение к github.com (github.com)|140.82.118.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/sjut/HSE-Compling/master/seminars/data/relations_with_concepts.csv [переход]
--2019-12-07 22:30:37--  https://raw.githubusercontent.com/sjut/HSE-Compling/master/seminars/data/relations_with_concepts.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.84.133
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.84.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 10359319 (9,9M) [text/plain]
Сохранение в: «relations_with_concepts.csv»

relations_with_conc 100%[===================>]   9,88M  8,14MB/s    за 1,2s    

2019-12-07 22:30:40 (8,14 MB/s) - «relations_with_

In [93]:
import csv

rels_list = []
with open("relations_with_concepts.csv", newline='', encoding='utf8') as rels:
    reader = csv.DictReader(rels, delimiter="\t")
    for row in reader:
        rels_list.append(row)


def get_supc2(concept_list, rels_list, has_up=True, depth=0, max_depth=-1):
    """
    Get list of all hypernym chains of the query
    - up a level
    - add all 'выше' concepts to list
    [[level_1, level_2.1, level_3.1], [level_1, level_2.2, level_3.2], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hypernyms
    :param has_up: (internal) bool(current top concept has a superconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of superconcept for every meaning of query
    """
    new_cl = concept_list[:]
    if (not has_up) or depth >= max_depth > 0:
        return new_cl
    has_up = False
    for chain in concept_list:
        index = new_cl.index(chain)
        word = chain[-1]
        for row in rels_list:
            new_chain = chain[:]
            if row['from'].lower() == word.lower() and row['relation'] == 'ВЫШЕ':
                new_chain.append(row['to'].lower())
                new_cl.insert(index + 1, new_chain)
                has_up = True
        if has_up:
            new_cl.remove(chain)
    return get_supc2(new_cl, rels_list, has_up, depth+1, max_depth)


def get_supc(concept_list, rels_list, has_up=True, depth=0, max_depth=-1):
    """
    Find list of all hypernyms of query by level down
    [[level_1], [level_2.1, level_2.2], [level_3.1, level_3.2, level_3.3], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hyponyms
    :param has_up: (internal) bool(current top concept has a subconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of subconcepts for every meaning of query
    """
    if (not has_up) or depth >= max_depth > 0:
        return concept_list
    has_up = False
    new_list = []
    for word in concept_list[-1]:
        for row in rels_list:
            if row['from'].lower() == word.lower() and row['relation'] == 'ВЫШЕ':
                if all(row['to'].lower() not in hypo for hypo in concept_list):
                    new_list.append(row['to'].lower())
                    has_up = True
    if has_up:
        concept_list.append(new_list)
    return get_supc(concept_list, rels_list, has_up, depth + 1, max_depth)


def get_subc2(concept_list, rels_list, has_down=True, depth=0, max_depth=-1):
    """
    Get list of all hyponym chains for word in query
    - down a level
    - add all 'ниже' concepts to list
    [[level_1, level_2.1, level_3.1], [level_1, level_2.2, level_3.2], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hyponyms
    :param has_down: (internal) bool(current top concept has a subconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of subconcepts for every meaning of query
    """
    new_cl = concept_list[:]
    if (not has_down) or depth >= max_depth > 0:
        return new_cl
    for chain in concept_list:
        has_down = False
        index = new_cl.index(chain)
        word = chain[-1]
        for row in rels_list:
            new_chain = chain[:]
            if row['from'].lower() == word.lower() and row['relation'] == 'НИЖЕ':
                new_chain.append(row['to'].lower())
                new_cl.insert(index + 1, new_chain)
                has_down = True
        if has_down:
            new_cl.remove(chain)
    return get_subc2(new_cl, rels_list, has_down, depth+1, max_depth)


def get_subc(concept_list, rels_list, has_down=True, depth=0, max_depth=-1):
    """
    Find list of all hyponyms of query by level down
    [[level_1], [level_2.1, level_2.2], [level_3.1, level_3.2, level_3.3], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hyponyms
    :param has_down: (internal) bool(current top concept has a subconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of subconcepts for every meaning of query
    """
    if (not has_down) or depth >= max_depth > 0:
        return concept_list
    has_down = False
    new_list = []
    for word in concept_list[-1]:
        for row in rels_list:
            if row['from'].lower() == word.lower() and row['relation'] == 'НИЖЕ':
                if all(row['to'].lower() not in hypo for hypo in concept_list):
                    new_list.append(row['to'].lower())
                    has_down = True
    if has_down:
        concept_list.append(new_list)
    return get_subc(concept_list, rels_list, has_down, depth+1, max_depth)

In [94]:
print(get_supc([["острый"]], rels_list))

print(get_supc2([["острый"]], rels_list))

print(get_subc([["острый"]], rels_list))

print(get_subc2([["острый"]], rels_list))

[['острый']]
[['острый']]
[['острый']]
[['острый']]


Тут чет ничего не нашлось.

In [97]:
from wiki_ru_wordnet import WikiWordnet

ruwn = WikiWordnet()
synsets = ruwn.get_synsets('острый')

In [98]:
for syn in synsets:
    for w in syn.get_words():
        print(w.lemma(), w.definition())
        print()

острый острый~ru~острый~ru~<!-- 3 --> {{п.}} имеющий [[жгучий]] вкус, раздражающий слизистую оболочку рта {{пример|{{выдел|Острый}} перец.}}~48016~88302

острый острый~ru~острый~ru~<!-- 8 --> [[критический]], [[катастрофичный]] {{пример|{{выдел|Острая}} нехватка средств при отсутствии доступных кредитов переросла в финансовый кризис.}}~15128~7782

острый острый~ru~острый~ru~<!-- 1 --> способный легко [[резать]] или [[прокалывать]] {{пример|{{выдел|Острый}} нож.}} {{пример|{{выдел|Острая}} игла.|}}~1067~84838



Нужные значения тут есть, это хорошо.

In [105]:
for syn in synsets:
    print(ruwn.get_hyponyms(syn), ruwn.get_hypernyms(syn))
    print()

set() set()

set() set()

set() set()



И тут ничего не нашлось. Не уверена, бывают ли 